<a href="https://colab.research.google.com/github/mohammedradman1/DSE_Term_Project/blob/main/DSE_Term_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtaining Data from Git Repository

### Obtain the data from the project repository in Github

In [1]:
# obtaining the data from drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# unzipping the file to cureent working directory
!unzip '/content/drive/MyDrive/DSE_Term_Project/TeePublic_review.zip' -d '/content'

Archive:  /content/drive/MyDrive/DSE_Term_Project/TeePublic_review.zip
replace /content/TeePublic_review.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Import needed libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # importing matplot library for graph
import seaborn as sns  # importing searborns library
import nltk # importing Natural Language Toolkit library
nltk.download('averaged_perceptron_tagger')

### Read Data into pandas datafram

In [ ]:
# Reading CSV file into pandas dataframe.
df = pd.read_csv("TeePublic_review.csv", encoding='latin-1') # reading the datafile
df.head()

# Exploratory Data Analysis

In [ ]:
# checking the shape of the data
print(df.shape)

In [ ]:
# viewing a sample of the data
df.sample(5)

In [ ]:
# viewing information of columns
df.info()

In [ ]:
# view statistical analysis of all numerical columns
df.describe()

In [ ]:
# Display the statistical analysis of numerical columns for target class (mean, std, min, max, etc).
df['review-label'].describe()

In [ ]:
# plotting the count of the unique values within the review-label
reviews = df['review-label']
ax = reviews.value_counts().sort_values()\
      .plot( title = 'Count of reviews ',
            figsize = (10,5),
             kind = 'bar')
ax.set_xlabel('Reviews stars')

In [ ]:
# checking for null values in columns
df.isnull().sum()

In [ ]:
df['review-label'].value_counts()

In [ ]:
example = df['review'].values[2]
print(example)

In [ ]:
tokens=nltk.wordpunct_tokenize(example)
tokens[ :10]

In [ ]:
tagged = nltk.pos_tag(tokens)
tagged[ :10]

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
entities = nltk.chunk.ne_chunk(tagged)
entities.pprint()

In [ ]:
#sentiment analysis by vader
#1.stop words are removed
#2.each word is scored and combined to a total score

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

In [ ]:
example

In [ ]:
sia.polarity_scores(example)



---



---



---



---



# Data Cleaning

## Filling with the mode per each category

In this method we as we have the rating value for each null review, <br> we can fill the review with the mode value of the rate category.

In [21]:
df.isnull().sum()

reviewer_id           1
store_location        0
latitude              0
longitude             0
date                  0
month                 0
year                  0
title                 9
review            30503
review-label          0
dtype: int64

In [22]:
# finding the modes for each category
rate_cat_modes = df.groupby('review-label')['review'].apply(lambda x: x.mode()[0])
rate_cat_modes

review-label
1    First time customer with TeePublic...placed an...
2    Did not look like what was pictured. The graph...
3    As described, good delivery I would use them a...
4    4 stars: Great ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ dec...
5                                        Great service
Name: review, dtype: object

In [23]:
df[ df['review'].isnull() ].sample(4)

,reviewer_id,store_location,latitude,longitude,date,month,year,title,review,review-label
12791,12791.0,US,37.090240,-95.712891,2023,4,2020 00:00:00,Nice soft shirt,NaN,5
4355,4355.0,SE,60.128161,18.643501,2023,5,2030 00:00:00,Good service and good produkts.,NaN,5
18145,18145.0,CA,56.130366,-106.346771,2023,3,2023 00:00:00,Incredible customer service!,NaN,5
248947,248947.0,US,37.090240,-95.712891,2019,7,2003 00:00:00,Great looking shirt!,NaN,5


In [24]:
# define a function that fills each Nan review with the most common value
def fillna_rate(row):
   row['review'] = rate_cat_modes[row['review-label']]
   return row

In [25]:
df_filled_mode = df.apply(fillna_rate, axis=1)

In [26]:
df_filled_mode.isnull().sum()

reviewer_id       1
store_location    0
latitude          0
longitude         0
date              0
month             0
year              0
title             9
review            0
review-label      0
dtype: int64

## Filling values using KNN

### Related Columns Preprocessing

In [74]:
# find null values
df.isnull().sum()

reviewer_id           1
store_location        0
latitude              0
longitude             0
date                  0
month                 0
year                  0
title                 9
review            30503
review-label          0
dtype: int64

we fill the null values of the other column so we can use it in the analysis. <br>
We will fill them with the mode value since they are small.

*   Title has 9 null values
*   reviewer_id will not be filled as it is not used in the KNN as a feature





In [75]:
# the mode value of the title
data = df
title_mode = data['title'].mode()[0]


We drop the unnecessary columns from the data:

*   Reviewer id: as it is a system generated value.
*   year: a duplicate of the date and consists of misleading data



In [76]:
# droping the reviewer_id and year from data
# store it in data again so that the droped column data is stored back
data = data.drop(['reviewer_id','year'], axis = 1)
# verify the drop
data.shape

(278100, 8)

In [77]:
# the title null value rows
data[ data['title'].isnull() ]

,store_location,latitude,longitude,date,month,title,review,review-label
12301,US,37.090240,-95.712891,2023,4,NaN,- These Simpsons themed t-shirts are the best ...,5
17903,CA,56.130366,-106.346771,2023,3,NaN,"-Ease of ordering has changed for the better, ...",4
47972,CA,56.130366,-106.346771,2022,11,NaN,"- offering a variety of products, by artisans-...",5
101144,US,37.090240,-95.712891,2022,4,NaN,NaN,4
124040,US,37.090240,-95.712891,2021,12,NaN,- Need confirmation numbers on a more timely b...,2
126610,DE,51.165691,10.451526,2021,12,NaN,+++fast shipping+++nice t-shirts+++very good+++,5
198331,CA,56.130366,-106.346771,2020,7,NaN,NaN,4
232304,US,37.090240,-95.712891,2019,12,NaN,"+Delivery time was great , printing was good b...",4
265332,US,37.090240,-95.712891,2018,12,NaN,NaN,5


In [78]:
# we fill the values of
data['title'].fillna( df['title'].mode()[0] , inplace = True)

In [79]:
# verify that the title column value is not null anymores
data.isnull().sum()

store_location        0
latitude              0
longitude             0
date                  0
month                 0
title                 0
review            30503
review-label          0
dtype: int64

In [80]:
# learning the data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278100 entries, 0 to 278099
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   store_location  278100 non-null  object 
 1   latitude        278100 non-null  float64
 2   longitude       278100 non-null  float64
 3   date            278100 non-null  int64  
 4   month           278100 non-null  int64  
 5   title           278100 non-null  object 
 6   review          247597 non-null  object 
 7   review-label    278100 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 17.0+ MB


we encode the store_location into numerical values.

In [81]:
# country encoding numbers
numerical_val = [x for x in range(1, 142)]
countries_name = data['store_location'].unique()

# replace the country with its label encoding.
data['store_location'].replace(countries_name, numerical_val, inplace = True)

data.head()

,store_location,latitude,longitude,date,month,title,review,review-label
0,1,37.090240,-95.712891,2023,6,Great help with lost order,I had an order that was lost in transit. When ...,5
1,1,37.090240,-95.712891,2023,6,I ordered the wrong size tee and hadï¿½ï¿½ï¿½,I ordered the wrong size tee and had difficult...,5
2,1,37.090240,-95.712891,2023,6,These guys offer the best customerï¿½ï¿½ï¿½,These guys offer the best customer service in ...,5
3,1,37.090240,-95.712891,2023,6,Good Stuff,Looked for an obscure phrase on a shirt. Teepu...,5
4,2,56.130366,-106.346771,2023,6,My order arrived in a good timelyï¿½ï¿½ï¿½,My order arrived in a good timely fashion & th...,4


In [82]:
data.shape

(278100, 8)

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278100 entries, 0 to 278099
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   store_location  278100 non-null  int64  
 1   latitude        278100 non-null  float64
 2   longitude       278100 non-null  float64
 3   date            278100 non-null  int64  
 4   month           278100 non-null  int64  
 5   title           278100 non-null  object 
 6   review          247597 non-null  object 
 7   review-label    278100 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 17.0+ MB


### Text Preprocessing

#### Importing Libraries and Cleaning data

We will import libraries for steming and stop words



In [84]:
import re  # For regular expressions
import numpy as np
# import the natural language processing library
import nltk
nltk.download('punkt') # download the punkt words tokens
from nltk.tokenize import word_tokenize  #$ importing the word tokenizer
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer  #$ importing the word lemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords  # For stop words
from nltk.stem import PorterStemmer  # For stemming

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
# function to clean the text columns
def clean_text(text):
    text = re.sub('<[^>]*>', '', text) #removing html tags
    text = re.sub('[^A-Za-z0-9\s]', '', text) # Remove punctuation and special characters
    text = re.sub('\s+', ' ', text) # Remove extra spaces
    text = text.lower()
    return text

In [86]:
# we need cleaning on title and review
data['title'] = data['title'].apply(clean_text)

#### Tokenization & Normalization

In [87]:
#converting the text to tokens and adding them into a new columns for the title
data['title_tokens'] = df['title'].apply(lambda text: word_tokenize(text) if isinstance(text, str) else [])

Word stemming and stop words removal, we return the words to their root and remove the stop words.

In [88]:
# we import the stopwords of English
stop_words = set( stopwords.words('english'))

In [89]:
# Normalizing and removal of stop words
stemmer = PorterStemmer()  # creating a stemmer

# stem and removal of stop words
def stemFun(tokens):
  stem_list = []
  for word in tokens:
    if word in stop_words:  # check if word is a stop word pass over it.
      pass
    else: # if word is not a stopword, stem it and store it in list.
      stem_list.append( stemmer.stem(word) )

  return  stem_list

data['title_tokens'] = data['title_tokens'].apply(stemFun)
# data['review_tokens'] = data['review_tokens'].apply(stemFun)

In [90]:
data.head()

,store_location,latitude,longitude,date,month,title,review,review-label,title_tokens
0,1,37.090240,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]"
1,1,37.090240,-95.712891,2023,6,i ordered the wrong size tee and had,I ordered the wrong size tee and had difficult...,5,"[i, order, wrong, size, tee, hadï¿½ï¿½ï¿½]"
2,1,37.090240,-95.712891,2023,6,these guys offer the best customer,These guys offer the best customer service in ...,5,"[these, guy, offer, best, customerï¿½ï¿½ï¿½]"
3,1,37.090240,-95.712891,2023,6,good stuff,Looked for an obscure phrase on a shirt. Teepu...,5,"[good, stuff]"
4,2,56.130366,-106.346771,2023,6,my order arrived in a good timely,My order arrived in a good timely fashion & th...,4,"[my, order, arriv, good, timelyï¿½ï¿½ï¿½]"


In [91]:
# importing necessary libraries
from gensim.models import Word2Vec

# getting the processed titles and reviews
processed_text_title = data[ 'title_tokens' ]

# Train Word2Vec model on the processed reviews
w2v_title_model = Word2Vec(processed_text_title.values.tolist(), min_count=1, window=5, vector_size=100)

In [92]:
# Function to create a document embedding by averaging word embeddings
def create_doc_embedding(doc):
    doc_embedding = 0
    words_count = 0
    for word in doc:
        if word in w2v_title_model.wv:
            doc_embedding += w2v_title_model.wv[word]
            words_count += 1
    if words_count != 0:
        doc_embedding = doc_embedding / words_count
    return doc_embedding

# Create document embeddings
def embed_doc (processed_text):
  doc_embeddings = []
  for text in processed_text.values:
      doc_embedding = create_doc_embedding(text)  # Using the previously defined function
      doc_embeddings.append(doc_embedding)

  return doc_embeddings

title_embeddings = embed_doc(processed_text_title)
# review_embeddings = embed_doc(processed_text_review, model_review )

In [93]:
# creating a dataframe with 100 columns
title_temp_df = pd.DataFrame([[ 0 for x in range(0,100)] for z in range(0,1)  ],
                       columns = ["title_embed_"+ str(x) for x in range(1,101)] )

# temparary dataframe to fill for missing values below
temp_df1 = pd.DataFrame([[ 0 for x in range(0,100)] for z in range(0,1)  ],
                       columns = ["title_embed_"+ str(x) for x in range(1,101)] )

In [94]:
title_temp_df.head()

,title_embed_1,title_embed_2,title_embed_3,title_embed_4,title_embed_5,title_embed_6,title_embed_7,title_embed_8,title_embed_9,title_embed_10,...,title_embed_91,title_embed_92,title_embed_93,title_embed_94,title_embed_95,title_embed_96,title_embed_97,title_embed_98,title_embed_99,title_embed_100
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
n= 1
# passing on the first n records
for i in range( len(title_embeddings)):
  title_embed_list = []

  if np.shape(title_embeddings[i]) == (100,):
   title_embed =  np.reshape( title_embeddings[i], (-1,100) ) # convert the shape to (1,100)

   title_embed_list =[[  x for x in title_embed [0] ]]  # convert to 2D list 1 row * 100 column

   # create a dataframe of the embedded title text
   embeding_df_temp = pd.DataFrame( title_embed_list ,
                          columns = title_temp_df.columns)
  else:
    embeding_df_temp = temp_df1

  title_temp_df = pd.concat([title_temp_df, embeding_df_temp]) # Add to temporary dataframe

  n +=1
  if n>10000:
    break


In [ ]:
# check the data
print(f"shape of title embedding dataframe: {title_temp_df.shape}")
# delete first line that was added only for temperay use
title_temp_df = title_temp_df.iloc[1:, :]
print(f"shape of title embedding dataframe after removal of 1st row: {title_temp_df.shape}")

### Adding Vectorized Text to data

In [ ]:
data.shape

In [ ]:
title_temp_df;
title_temp_df.shape

In [ ]:
data.shape

In [ ]:
# adding the embedding back to the data
data = data.merge(title_temp_df, how='left', right_index = True, left_index = True)

In [ ]:
# viewing the addition of the new data columns
data.head()

### Numerical Preprocessing

In [ ]:
# we perform scaling for the numerical columns and the target class
from sklearn.preprocessing import StandardScaler  # import standard scalar

# Extract numerical columns
numerical_cols = ["store_location", "latitude", "longitude", "date", "month"]
numerical_data = data[numerical_cols]

# Standardize numerical data which is often recommended for KNN
scaler = StandardScaler()
numerical_data_scaled = pd.DataFrame(scaler.fit_transform(numerical_data),
                                      columns = numerical_data.columns)

In [ ]:
# adding scaled values to the data
data = data.join(numerical_data_scaled, rsuffix='_scaled');  # using semicolon to prevent printing result
data.head(5)

### Splitting the data for **KNN**

The columns that will be used for analysis are:

*  `["title_embed_"+ str(x) for x in range(1,101)]`
*   Plus numerical scaled columns



In [ ]:
# name of features for the embedded title
features_name = ["title_embed_"+ str(x) for x in range(1,101)]
for index, name in enumerate(numerical_data_scaled.columns):
  # skip the last index name which is the target variable
  if index != len(numerical_data_scaled.columns)-1:
    features_name.append(name + "_scaled") # add the name of the scaled feature.
  else:  # assigning the name for target variable
    target_name = name


In [ ]:
data

In [59]:
# temp until we convert entire data into the dataframe array
data2 = data
missing_review_data = data2[ data2['review'].isnull() ] # store the missing data separately
data = data.dropna() # remove null values as

In [60]:
data['title_embed_1'].isnull().sum()

0

In [61]:
missing_review_data['title_embed_1'].isnull().sum()

30503

In [62]:
data['review'].isnull().sum()

0

In [63]:
print(f"data shape: {data.shape} ")
print(f"missing reviews data shape: {missing_review_data.shape}")

data shape: (10000, 114) 
missing reviews data shape: (30503, 114)


In [64]:
data

,store_location,latitude,longitude,date,month,title,review,review-label,title_tokens,title_embed_1,...,title_embed_96,title_embed_97,title_embed_98,title_embed_99,title_embed_100,store_location_scaled,latitude_scaled,longitude_scaled,date_scaled,month_scaled
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",0.037150,...,0.391522,0.634187,-0.422079,0.225698,-0.067162,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",0.384588,...,0.347595,0.914932,-0.494232,0.479215,-0.388568,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",-0.488505,...,0.193130,0.346259,-0.166887,-0.186420,-0.246468,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",-0.422302,...,-0.411632,0.116647,-0.888758,0.912585,-0.459413,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",-0.162895,...,0.185182,0.105389,-0.120488,0.463756,-0.161284,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",0.031488,...,-0.608782,-0.187484,0.126868,0.649345,0.141329,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",-0.189940,...,-0.177374,0.065937,-0.128920,0.263576,0.471110,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",-0.422302,...,-0.411632,0.116647,-0.888758,0.912585,-0.459413,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
0,1,37.09024,-95.712891,2023,6,great help with lost order,I had an order that was lost in transit. When ...,5,"[great, help, lost, order]",0.145652,...,0.210193,0.223963,-0.360403,0.126311,-0.463845,-0.133464,-0.009546,-0.199667,1.434708,-0.318703


In [65]:
# obtain the features + the target variable
X, Y = data[features_name], data[target_name]

In [66]:
# splitting the data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

In [67]:
# Creating the model
from sklearn.neighbors import KNeighborsClassifier
# create an instance of the classifier
k_model = KNeighborsClassifier(n_neighbors = 3)


In [68]:
# Model training
k_model.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=3)

In [69]:
# Test model
predicted = k_model.predict( X_test)

In [70]:
# measuring accuracy
from sklearn.metrics import accuracy_score

accur = accuracy_score(Y_test, predicted)
print("KNN accuracy: ", accur)

KNN accuracy:  1.0


In [71]:
# Choose a target review
target_review_index = 10  # assume we wish to find it for this review
target_combined_data = data.iloc[target_review_index,:][features_name]

# Find the closest neighbors
distances, indices = k_model.kneighbors([target_combined_data])

# Print the closest reviews
for i in range(len(indices[0])):
    neighbor_index = indices[0][i]
    # print(f"Closest Review {i+1}:\n{data.iloc[neighbor_index]}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [72]:
missing_review_data

,store_location,latitude,longitude,date,month,title,review,review-label,title_tokens,title_embed_1,...,title_embed_96,title_embed_97,title_embed_98,title_embed_99,title_embed_100,store_location_scaled,latitude_scaled,longitude_scaled,date_scaled,month_scaled
121,1,37.090240,-95.712891,2023,6,awesome tshirt love it vintage,NaN,5,"[awesom, t-shirt, love, vintag, ï¿½ï¿½ï¿½ï¿½]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
128,1,37.090240,-95.712891,2023,6,nice tee quick delivery,NaN,5,"[nice, tee, quick, deliveri, -]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
130,1,37.090240,-95.712891,2023,6,big selections lots to choose from,NaN,5,"[big, select, lot, choos]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
142,1,37.090240,-95.712891,2023,6,great company,NaN,5,"[great, compani]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
146,2,56.130366,-106.346771,2023,6,great as always,NaN,5,"[great, alway, !]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,1.434708,-0.318703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278058,1,37.090240,-95.712891,2018,4,awsome quality quick delivery,NaN,5,"[awsom, qualiti, quick, deliveri]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,-2.086508,1.338558
278063,1,37.090240,-95.712891,2018,4,i was impressed,NaN,5,"[i, impress, !]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,-2.086508,1.338558
278064,2,56.130366,-106.346771,2018,4,i love my shirt,NaN,5,"[i, love, shirt, !]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,-2.086508,1.338558
278074,1,37.090240,-95.712891,2018,4,killer shirts,NaN,5,"[killer, shirt]",NaN,...,NaN,NaN,NaN,NaN,NaN,-0.133464,-0.009546,-0.199667,-2.086508,1.338558


In [ ]:
# Now we fill the missing values using the KNN
missing_rows_indices = missing_review_data.index.values.tolist()
# missing_rows_indices = [3,3]

# we pass per each record and find its neighbors
for index in missing_rows_indices:
  target_combined_data = missing_review_data.iloc[index,:][features_name]
  # Find the closest neighbors
  print(target_combined_data)
  distances, indices = k_model.kneighbors([target_combined_data])
  # closest neighbor index
  neighbor_index = indices[0][0]

  # closed neighbor review
  review = data.iloc[neighbor_index, :]['review']

  # store the review in the missing data
  missing_review_data.iloc[index, :]['review'] = review


In [ ]:
# add the missing_review_data to the entire data
df_filled_knn = pd.concat(data, missing_review_data)

In [ ]:
# check for any missing values
df_filled_knn.isnull().sum()

# Complete Data

Here are the completed data for filled by both the mode per category and the KNN model.

In [ ]:
# Data Filled Using Mode
df_filled_mode

# Data Filled Using KNN Model
df_filled_knn